### 3 subword algorithms help to improve your NLP model performance
- Byte Pair Encoding (BPE)
- WordPiece
- Unigram Language Model
- SentencePiece  

Subword balances vocabulary size and footprint. Extreme case is we can only use 26 token (i.e. character) to present all English word. 16k or 32k subwords are recommended vocabulary size to have a good result.

Many Asian language word cannot be separated by space. Therefore, the initial vocabulary is larger than English a lot. You may need to prepare over 10k initial word to kick start the word segmentation. From Schuster and Nakajima research, they propose to use 22k word and 11k word for Japanese and Korean respectively.  
https://medium.com/@makcedward/how-subword-helps-on-your-nlp-model-83dd1b836f46

To perform subword tokenization, BPE is slightly modified in its implementation such that the frequently occurring subword pairs are merged together instead of being replaced by another byte to enable compression. This would basically lead the rare word athazagoraphobia to be split up into more frequent subwords such as ['▁ath', 'az', 'agor', 'aphobia'].
https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10

### Tokenizers: How machines read - 28 JANUARY 2020
Recommended read on tokenization  

- **BPE**: Just uses the frequency of occurrences to identify the best match at every iteration until it reaches the predefined vocabulary size.
- **WordPiece**: Similar to BPE and uses frequency occurrences to identify potential merges but makes the final decision based on the likelihood of the merged token
- **Unigram**: A fully probabilistic model which does not use frequency occurrences. Instead, it trains a LM using a probabilistic model, removing the token which improves the overall likelihood the least and then starting over until it reaches the final token limit.
- **SentencePiece** basically tries to bring all the subword tokenization tools and techniques under one banner. _" SentencePiece is a re-implementation of sub-word units, an effective way to alleviate the open vocabulary problems in neural machine translation. SentencePiece supports two segmentation algorithms, byte-pair-encoding (BPE) [Sennrich et al.] and unigram language model [Kudo.]. "_ (BPE and Unigram are reimplemented with improvements).
    - __All other models assume input is already tokenized__: BPE and Unigram are great models but they share one big disadvantage- they both need to have their input already tokenized. BPE needs to have the input tokenized so that every character (including word-boundary characters) are tokenized. Only then can BPE count frequencies and start to merge tokens. Usually this is done by simply doing word level tokenization but, as we discussed earlier, this is a problem with tokenization since not all languages are space segmented. Similarly, the unigram model needs to have its input tokenized before it can start discarding tokens based on their probability distribution. SentencePiece deals with this by simply taking in an input in raw text and then doing everything (which we will discuss below) needed on that input to perform subword tokenization.
    - __Encode everything as unicode ...__: SentencePiece first converts all the input into unicode characters. This means it doesn’t have to worry about different languages or characters or symbols. If it uses unicode it can just treat all input in the same way, which allows it to be language agnostic
    - __… including  the spaces__: To get around the word segmenting issues, SentencePiece simply encodes spaces as a unicode symbol. Specifically it encodes it as unicode value U+2581 (underscore ‘_’ to those of us who don’t speak unicode). This helps with the language agnostic issues and the decoding issue. Since spaces are unicode encoded then they can be easily reversed or decoded and treated (i.e learned) like a normal language character. It sounds like a simple approach and I guess it is, but the best ideas tend to seem that way in the end


https://blog.floydhub.com/tokenization-nlp/

### __Huggingface `tokenizers`__ : 
Provided Tokenizers
- CharBPETokenizer: The original BPE
- ByteLevelBPETokenizer: The byte level version of the BPE
- SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
- BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece  
 
We designed the library so that it provides all the required blocks to create end-to-end tokenizers in an interchangeable way. In that sense, we provide
these various components: 

- **Normalizer**: Executes all the initial transformations over the initial input string. For example when you need to
lowercase some text, maybe strip it, or even apply one of the common unicode normalization process, you will add a Normalizer. 
- **PreTokenizer**: In charge of splitting the initial input string. That's the component that decides where and how to
pre-segment the origin string. The simplest example would be like we saw before, to simply split on spaces.
- **Model**: Handles all the sub-token discovery and generation, this part is trainable and really dependant
 of your input data.
- **Post-Processor**: Provides advanced construction features to be compatible with some of the Transformers-based SoTA
models. For instance, for BERT it would wrap the tokenized sentence around [CLS] and [SEP] tokens.
- **Decoder**: In charge of mapping back a tokenized input to the original string. The decoder is usually chosen according
to the `PreTokenizer` we used previously.
- **Trainer**: Provides training capabilities to each model. 

Notebook for Tokenizers: https://github.com/huggingface/transformers/blob/master/notebooks/01-training-tokenizers.ipynb  
Github Link for Python Binding: https://github.com/huggingface/tokenizers/tree/master/bindings/python

Implementation: https://github.com/huggingface/tokenizers/tree/master/bindings/python/tokenizers/implementations


## Imports

In [1]:
!pip install tokenizers

In [64]:
import os
from pathlib import Path
from collections import Counter

## Constants

In [4]:
DATA_PATH = Path("../data")

# DATA_RAW_PATH = DATA_PATH/"raw"
DATA_RAW_EXTRACTED_PATH = DATA_PATH/"raw_data_extraction"

# 1. The data from thwiki
THWIKI_FOLDER = Path("thwiki-20200601-extracted")
WIKI_FILES = list((DATA_RAW_EXTRACTED_PATH/THWIKI_FOLDER).glob("Wiki*.txt"))
list(map(print , WIKI_FILES[:5]))


# 2. The classification data from jung and ninja
CLASSIFICATION_JUNG_NINJA_FOLDER = Path("classification_dataset")
CLASSIFICATION_FILES = list((DATA_RAW_EXTRACTED_PATH/CLASSIFICATION_JUNG_NINJA_FOLDER).glob("*.txt"))
list(map(print , CLASSIFICATION_FILES[:5]))

# 3. The Data from p'Moo Crawlers
ANOTHER_WEBSITE_MOO_FOLDER = Path("another_website")
ANOTHER_WEBSITE_FILES = list((DATA_RAW_EXTRACTED_PATH/ANOTHER_WEBSITE_MOO_FOLDER).glob("*.txt"))
list(map(print , ANOTHER_WEBSITE_FILES[:5]))


# 4. Senior Project Files
SENIOR_PROJ_FOLDER = Path("data_lm")
SENIOR_PROJ_FILES = list((DATA_RAW_EXTRACTED_PATH/SENIOR_PROJ_FOLDER).glob("*.txt"))
list(map(print , SENIOR_PROJ_FILES[:5]))

# 5. Guru Crawler Files
GURU_CRAWLER_FOLDER = Path("social_listening")
GURU_CRAWLER_FILES = list((DATA_RAW_EXTRACTED_PATH/GURU_CRAWLER_FOLDER).glob("*.txt"))
list(map(print , GURU_CRAWLER_FILES[:5]))

ALL_FILES = WIKI_FILES + CLASSIFICATION_FILES + ANOTHER_WEBSITE_FILES + SENIOR_PROJ_FILES + GURU_CRAWLER_FILES
print(f"\nI have a total of {len(ALL_FILES)} files!")


# Output is in bytes - helper from Pathlib Path https://stackoverflow.com/questions/2104080/how-can-i-check-file-size-in-python
def getStat(prev_value, cur_value):
    if isinstance(prev_value, int):
        return prev_value + cur_value.stat().st_size
    return prev_value.stat().st_size + cur_value.stat().st_size

from functools import reduce
print(f"Amounts to a total of {reduce(getStat, ALL_FILES)/1e6:.2f} MB")

../data/raw_data_extraction/thwiki-20200601-extracted/WikiAD_2.txt
../data/raw_data_extraction/thwiki-20200601-extracted/WikiAC_2.txt
../data/raw_data_extraction/thwiki-20200601-extracted/WikiAF_2.txt
../data/raw_data_extraction/thwiki-20200601-extracted/WikiAE_1.txt
../data/raw_data_extraction/thwiki-20200601-extracted/WikiAB_1.txt
../data/raw_data_extraction/classification_dataset/siamrath_0.txt
../data/raw_data_extraction/classification_dataset/dailynews_0.txt
../data/raw_data_extraction/classification_dataset/prachachat_0.txt
../data/raw_data_extraction/classification_dataset/naewna_0.txt
../data/raw_data_extraction/classification_dataset/springnews_0.txt
../data/raw_data_extraction/another_website/pantip_87.txt
../data/raw_data_extraction/another_website/pantip_535.txt
../data/raw_data_extraction/another_website/dailynews_15.txt
../data/raw_data_extraction/another_website/pantip_219.txt
../data/raw_data_extraction/another_website/pantip_549.txt
../data/raw_data_extraction/data_lm/

In [5]:
[file for file in list(map(str, ALL_FILES)) if os.path.isdir(file)]

[]

In [6]:
# import chardet
# for filename in list(map(str, ALL_FILES))[::-1]:
#     with open(filename, 'rb') as f:
#         content_bytes = f.read()
#     detected = chardet.detect(content_bytes)
#     encoding = detected['encoding']
#     print(f"{filename}: detected as {encoding}.")

In [7]:
list((DATA_RAW_EXTRACTED_PATH/CLASSIFICATION_JUNG_NINJA_FOLDER).glob("*"))

[PosixPath('../data/raw_data_extraction/classification_dataset/siamrath_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/dailynews_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/prachachat_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/naewna_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/springnews_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/pptv36_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/prbangkok_0.txt'),
 PosixPath('../data/raw_data_extraction/classification_dataset/thaipbs_0.txt')]

# Train `SeniorProjectTokenizer` 


In [16]:
from senior_project_util import ThaiTokenizer, pre_rules_th, post_rules_th
from fastai.text.transform import BaseTokenizer, Tokenizer
from fastai.text.data import TokenizeProcessor, NumericalizeProcessor

In [22]:
text='ม่ายเอาเปงไงบ้างน่ารักจุงเบย'
pyThai_tt = ThaiTokenizer()
a = pyThai_tt.tokenizer(text)
a

['ม่าย', 'เอา', 'เปง', 'ไง', 'บ้าง', 'น่ารัก', 'จุงเบย']

In [44]:
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th, n_cpus=10)
tt.process_all([text[:1000]])

In [47]:
tokenizer_processor = TokenizeProcessor(tokenizer=tt, chunksize=300000, mark_fields=False)

In [46]:
t_processor.process_one("เวลา 12.00น.")

['xxbos', ' ', 'เวลา', 'xxnum', ' ', 'น.']

In [81]:
def load_data_tokenized(file_path):
    print(f"I AM DOING {file_path}")
    directory, filename = os.path.split(file_path)
#     cached_features_file = os.path.join(
#         self.cached_directory, f"cached_lm_{tokenizer.__class__.__name__}_{str(self.block_size)}_{filename}",
#     )

    with open(file_path, encoding="utf-8") as f:
        text = f.read()
    tokens = tokenizer_processor.process_one(text)

    return tokens

In [82]:
from multiprocessing import Pool
import tqdm

num_processes = 10
sample_path = ALL_FILES

with Pool(processes=num_processes) as p:
    tokens = list(tqdm.tqdm(p.imap(load_data_tokenized, sample_path), total=len(sample_path)))

  0%|          | 0/1409 [00:00<?, ?it/s]

I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAC_2.txtI AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAF_2.txtI AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAE_1.txtI AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAD_2.txt
I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAB_1.txtI AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAA_3.txt

I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAE_0.txt
I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAE_3.txtI AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAC_0.txtI AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAB_2.txt





I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAF_0.txt
I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAD_3.txt


Process ForkPoolWorker-91:
Process ForkPoolWorker-84:
Process ForkPoolWorker-92:
Process ForkPoolWorker-88:
Process ForkPoolWorker-89:
Process ForkPoolWorker-86:
Process ForkPoolWorker-90:
Process ForkPoolWorker-85:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _b

  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/tcc.py", line 66, in tcc
    m = _PAT_TCC.match(text[p:])
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/newmm.py", line 75, in _onecut
    valid_poss = tcc_pos(text)  # breaking positions that are TCC-valid
  File "/opt/conda/lib/python3.7/site-packages/fastai/text/transform.py", line 106, in process_text
    toks = tok.tokenizer(t)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/site-packages/fastai/text/data.py", line 291, in process_one
    return self.toke

  File "/opt/conda/lib/python3.7/site-packages/fastai/text/transform.py", line 106, in process_text
    toks = tok.tokenizer(t)
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/tcc.py", line 66, in tcc
    m = _PAT_TCC.match(text[p:])
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/tcc.py", line 66, in tcc
    m = _PAT_TCC.match(text[p:])
KeyboardInterrupt
KeyboardInterrupt
  File "/workdir/Code/bma_transformer_model/thai-lm-training/senior_project_util.py", line 84, in tokenizer
    return self.pyengine.word_tokenize(t)
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/core.py", line 493, in word_tokenize
    keep_whitespace=self.__keep_whitespace,
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/tcc.py", line 88, in tcc_pos
    for w in tcc(text):
  File "/opt/conda/lib/python3.7/site-packages/pythainlp/tokenize/core.py", line 493, in word_tokenize
    keep_whitespace=self.__keep_whitespace,
  File "<ipython-input-81

KeyboardInterrupt: 

In [62]:
len(tokens)

10

In [55]:
load_data_tokenized(ALL_FILES[0])[:10]

I AM DOING ../data/raw_data_extraction/thwiki-20200601-extracted/WikiAD_2.txt


['xxbos', ' ', 'ขั้นตอนวิธี', 'ของ', 'ค', 'ริ', 'ส', 'โต', 'ไฟ', 'ด์']

## Making Vocabs
copied from Fastai [`Vocab.create()`](https://github.com/fastai/fastai/blob/d418294f0f17382f7a33bb72b93f5055a7768b14/fastai/text/transform.py#L149)

In [76]:
max_vocab = 50000
min_freq = 3

BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_REP,TK_WREP, TK_NUM, TK_LAUGH = 'xxrep','xxwrep', 'xxnum', 'xxlaugh'
text_spec_tok = [UNK,PAD,BOS,EOS,FLD,TK_REP,TK_WREP, TK_NUM, TK_LAUGH]

In [65]:
freq = Counter(p for o in tokens for p in o)

In [72]:
print({i:v for i,v in freq.most_common(20)})

{' ': 56922, 'ใน': 4840, 'และ': 4410, '\n': 4062, 'ที่': 3813, 'เป็น': 3394, 'ของ': 3071, 'ได้': 2150, 'มี': 1895, '(': 1654, ')': 1569, '"': 1534, 'ปี': 1273, 'ให้': 1194, 'โดย': 1182, 'พ.ศ.': 1159, 'จาก': 1104, 'กับ': 1023, 'ซึ่ง': 1001, 'ทํา': 994}


In [73]:
itos = [o for o,c in freq.most_common(max_vocab) if c >= min_freq]
print(itos[:100])

[' ', 'ใน', 'และ', '\n', 'ที่', 'เป็น', 'ของ', 'ได้', 'มี', '(', ')', '"', 'ปี', 'ให้', 'โดย', 'พ.ศ.', 'จาก', 'กับ', 'ซึ่ง', 'ทํา', 'การ', 'จะ', 'เมื่อ', 'ว่า', 'นี้', 'ไป', 'มา', ',', '-', 'หรือ', 'วันที่', 'ด้วย', 'คือ', 'ก็', 'คน', 'พระ', 'แต่', 'ได้รับ', 'เพื่อ', 'เรื่อง', 'ใช้', 'เขา', 'เมือง', 'ถูก', 'น้ํา', 'จํา', 'ํา', 'ยัง', 'จึง', 'อยู่', '2', 'ค.ศ.', 'ไม่', 'นํา', 'สํา', '1', 'ทาง', 'ผู้', 'นั้น', 'แห่ง', 'ขึ้น', 'ที่มี', '์', 'เกิด', 'สามารถ', 'หนึ่ง', 'แล้ว', '3', 'ถึง', 'คํา', 'ภาพยนตร์', 'ความ', 'ทั้ง', 'หรับ', 'ชื่อ', 'ส', 'แบบ', 'กา', 'ภาษา', 'ต่อมา', '()', 'อีก', 'กัน', 'ปัจจุบัน', 'ระหว่าง', 'เช่น', 'ทรง', 'เธอ', 'อร', 'ตัว', 'ไทย', '.', 'ตาม', 'ประเทศ', 'ด้าน', 'เพลง', 'มาก', 'จัด', 'อยู่ใน', 'เข้า']


In [80]:
for o in reversed(text_spec_tok):
    if o in itos: itos.remove(o)
    itos.insert(0, o)
itos = itos[:max_vocab]
if len(itos) < max_vocab: #Make sure vocab size is a multiple of 8 for fast mixed precision training
    while len(itos)%8 !=0: itos.append('xxfake')
print("ITOS", itos[:1000])

ITOS ['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxnum', 'xxlaugh', ' ', 'ใน', 'และ', '\n', 'ที่', 'เป็น', 'ของ', 'ได้', 'มี', '(', ')', '"', 'ปี', 'ให้', 'โดย', 'พ.ศ.', 'จาก', 'กับ', 'ซึ่ง', 'ทํา', 'การ', 'จะ', 'เมื่อ', 'ว่า', 'นี้', 'ไป', 'มา', ',', '-', 'หรือ', 'วันที่', 'ด้วย', 'คือ', 'ก็', 'คน', 'พระ', 'แต่', 'ได้รับ', 'เพื่อ', 'เรื่อง', 'ใช้', 'เขา', 'เมือง', 'ถูก', 'น้ํา', 'จํา', 'ํา', 'ยัง', 'จึง', 'อยู่', '2', 'ค.ศ.', 'ไม่', 'นํา', 'สํา', '1', 'ทาง', 'ผู้', 'นั้น', 'แห่ง', 'ขึ้น', 'ที่มี', '์', 'เกิด', 'สามารถ', 'หนึ่ง', 'แล้ว', '3', 'ถึง', 'คํา', 'ภาพยนตร์', 'ความ', 'ทั้ง', 'หรับ', 'ชื่อ', 'ส', 'แบบ', 'กา', 'ภาษา', 'ต่อมา', '()', 'อีก', 'กัน', 'ปัจจุบัน', 'ระหว่าง', 'เช่น', 'ทรง', 'เธอ', 'อร', 'ตัว', 'ไทย', '.', 'ตาม', 'ประเทศ', 'ด้าน', 'เพลง', 'มาก', 'จัด', 'อยู่ใน', 'เข้า', 'นา', 'ต่อ', 'แสดง', 'จํานวน', 'วัด', 'สร้าง', 'ริ', 'กว่า', 'ช่วง', ':', 'หลาย', 'ร์', 'รา', 'ส่วน', 'ชุด', 'แรก', 'ๆ', 'โรงเรียน', 'อําเภอ', '4', 'นัก', 'ระดับ', 'ครั้ง', 'ตั้งแต่

In [79]:
len(itos)

8064

### Write out the itos

In [ ]:
WRITE_OUT_FILE = "senior_project_vocab.txt"
with open(WRITE_OUT_FILE, 'w', encoding='utf-8') as f:
    f.writelines(itos)
print(f"Successfully written vocabulary itos in {WRITE_OUT_FILE}")

In [27]:
text[:1000]

'ขั้นตอนวิธีของคริสโตไฟด์\nขั้นตอนวิธีของคริสโตไฟด์ () ตั้งชื่อตาม นิคอส คริสโตฟิลด์ เป็นขั้นตอนวิธีในการแก้ปัญหาบางกลุ่มของปัญหาการเดินทางของพนักงานขาย ที่มีเส้นเชื่อมถ่วงน้ําหนักเป็นไปตามความไม่เสมอภาคของสามเหลี่ยม ซึ่งได้คําตอบที่มีอัตราส่วนการประมาณ เป็น 1.5 เท่าของคําตอบดีที่สุด\nขั้นตอนที่ 1: สร้าง ต้นไม้ทอดข้ามที่น้อยที่สุด formula_6 จาก formula_2\nขั้นตอนที่ 2: ให้ formula_8 เป็นเซตของจุดยอดที่มี ระดับขั้น เป็นจํานวนคี่ ใน formula_6 และหา การจับคู่สมบูรณ์ formula_10 ซึ่งมีน้ําหนักน้อยที่สุดใน กราฟบริบูรณ์ บนจุดยอดใน formula_8\nขั้นตอนที่ 3: รวมเส้นเชื่อมของ formula_10 และ formula_6 เป็น มัลติกราฟ formula_14\nขั้นตอนที่ 4: สร้างวงจรออยเลอร์ ใน formula_14\nขั้นตอนที่ 5: สร้างวงจรแฮมิลตัน จากขึั้นตอนที่แล้วโดยข้ามจุดยอดที่เยี่ยมชมแล้วออกไป ("shortcutting")\nผลลัพธ์ของขั้นตอนวิธีนี้มีค่าเป็น 1.5 เท่าของของคําตอบดีที่สุด\nพิสูจน์ได้ดังนี้:\nให้ formula_16 แทนเซตของเส้นเชื่อมของคําตอบดีสุดของปัญหาการเดินทางของพนักงานขาย สําหรับformula_2, เนื่องจากformula_18 เชื่อมต่อกันบริบูรณ์ จึงมี

## Special Tokens
> SentencePiece reserves vocabulary ids for special meta symbols, e.g., unknown symbol (<unk\>), BOS (<s\>), EOS (</s\>) and padding (<pad\>). Their actual ids are configured with command line flags. We can also define custom meta symbols to encode contextual information as virtual tokens. Examples include the language- indicators, <2ja> and <2de>, for multilingual models

-- From SentencePiece Paper

Bert uses the special tokens `[UNK] [CLS] [SEP] [PAD] [MASK]`

- Unknown: `[UNK]` `<unk>`
- Beginning of Sentence (BOS): `[CLS]` `<s>`
- Ending of Sentence (EOS): `[SEP]` `</s>`
- Padding: `[PAD]`  `<pad>`
- Mask: `[MASK]` `<mask>`

In [ ]:
tokenizer.train(files=list(map(str, ALL_FILES)), 
                vocab_size=30522, 
                min_frequency=2,
                show_progress=True,
                special_tokens=["<s>","<pad>","</s>","<unk>","<mask>"],
               )
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

In [ ]:
!mkdir all-data-wordpiece-30522

In [1]:
# And finally save it somewhere
# ! mkdir all-data-bytebpe-30522
tokenizer.save_model("all-data-wordpiece-30522")
tokenizer.save("./all-data-wordpiece-30522.tokenizer.json", pretty=True)

NameError: name 'tokenizer' is not defined

### Test Tokenize on Pantip Sample ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ
https://pantip.com/topic/40006922

In [ ]:
encoded = tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
print(encoded.ids)
print(encoded.tokens)
print(list(map(lambda x : tokenizer.decode([x]), encoded.ids)))

In [ ]:
encoded = tokenizer.encode(u"Hello Thisis a test in English. How is this algorithm learning?? I dunno as well.")
print(encoded.ids)
print(encoded.tokens)

### Test Tokenize on Pantip Sample ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ
https://pantip.com/topic/40006922

In [ ]:
text = "ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ  ก่อนอื่นผมต้องบอกก่อนเลยว่าคนเราจะเลือกกินอาหารแบบไหนชอบแบบไหนเป็นเรื่องของความชอบส่วนตัวนะครับทุกคนมีสิทธิในการเลือกของที่ชอบและไม่ชอบอยู่แล้ว แต่ผมรู้สึกว่าตอนนี้ผมกำลังประสบปัญหาที่ดูเหมือนจะเล็กแต่กลายเป็นว่ามันค่อนข้างใหญ่ ผมคบกับแฟนมา6ปีแล้วครับ ผมเป็นคนชอบกินอาหารญี่ปุ่นและปลาดิบแต่แฟนผมไม่กินปลาดิบเลย ผมอยากกินบุฟเฟ่เนื้อแต่แฟนผมก็ไม่กินเนื้อ เราเลยไม่ได้เข้าทานร้านบุฟเฟ่เนื้อและบุฟเฟ่อาหารญี่ปุ่นกันเพราะรู้สึกลัวแฟนผมทานไม่คุ้ม และเรื่องใหญ่เลยคือผมเป็นคนชอบทานอาหารรสจัดและรสเผ็ดมาก แต่แฟนผมทานเผ็ดไม่ได้เลยเวลาเราไปกินส้มตำกันก็จะสั่ง ส้มตำไม่ใส่พริก ต้มแซ่บไม่ใส่พริก ลาบไม่ใส่พริก ร้านกับข้าวอื่นๆก็เช่นกันแฟนผมจะไม่ชอบกินผักไม่ค่อยสั่งกับข้าวที่เป็นผักแล้วผมชอบผักบุ้งทอดกรอบ เห็ดหอมสดทอดมาก แต่ก็ไม่ได้สั่งเพราะว่าเธอไม่กินถึงเค้าจะบอกให้สั่งเลยๆก็เถอะแต่ผมก็ยังเกรงใจเธออยู่ดีอ่ะครับ ผมรู้สึกกินอาหารไม่มีความสุขเลยชีวิตผมขาดรสเผ็ดไปเหมือนจะขาดใจเหมือนมันทำให้ขาดความสุขไปอย่างนึงเลยอ่ะครับ ยิ่งถ้าเราแต่งงานกันแล้วผมก็อาจจะต้องมีปัญหาเรื่องนี้มากขึ้น พอผมเห็นคู่ที่ชอบทานอาหารเหมือนๆกันเห็นเค้ากินอาหารกันอย่างมีความสุขแล้วผมรู้สึกอิจฉามากๆเลย มีใครเคยมีปัญหาแบบผมมั้ยครับแล้วจะแก้ปัญหานี้ยังไงดีครับ"
encoded = tokenizer.encode(text)
print(encoded.ids)
print(encoded.tokens)
print(list(map(lambda x : tokenizer.decode([x]), encoded.ids)))

### Test Tokenize on Pantip Sample อาการแบบนี้คือไรกัน?
https://pantip.com/topic/40009518

In [ ]:
text = "อาการแบบนี้คือไรกัน?  เขาคุยกับเรามา 5-6 เดือน เราตามจีบเขานะคะ ก็คุยกันมา ในระยะเวลาเขาบอกว่า ถ้าเราลด นน เพื่อเขาได้ เขาจะยอมเป็นแฟนเรา ตรรกะโง่มากนะคะ แต่ถามว่าทำมั้ย ทำค่ะ พอไปรู้ว่าเขาคุยกับเพื่อน เพื่อนเขาถามว่า รู้สึกยังไงกับเรา เขาตอบเพื่อนว่า เขาว่าเขาควรอยู่คนเดียว ยังไม่พร้อมจะรักใคร จนตอนนี้เราเริ่มรู้สึกว่า ทำไมเราต้องทำขนาดนั้น ถ้าเขาจะรัก รักที่เป็นตัวเราไม่ได้หรอ หลังๆเลยเริ่มสนใจเขาน้อยลง แต่ยังคุยกันเหมือนเดิม เราลองแกล้งเงียบไป ไม่ทักไปครึ่งวัน ปกติเราจะมีการมอนิ่งกันตอนเช้าค่ะ พอเราไม่ทักไป เขาทำงานเสร็จ ถึงเวลาพักของเขา เขาก็ทักมาว่า กินข้าวกัน เราก็ยิ่ง งง ก็คิดว่า เขาอาจจะชินหับการคุยกับเราทุกวันเฉยๆ นี่เลยไม่ได้สนใจในส่วนนั้น เราก็ตอบตามปกติ จนเมื่อคืนมีคนมาทักเราจีบเรา จะไปส่งเราที่บ้าน เราก็เลยเล่าให้เขาฟังว่า ให้ไลน์ไป ให้เขาไปส่งอยู่แต่ไมไ่ด้นั่งรถคันเดียวกัน เราก็ขับของเรา คนที่มาจีบเราเขาก็ขับคันของเขาแค่มาส่งเฉยๆ พอเช้ามาเขาทักมามอนิ่ง ก็ถามเราเรื่องเมื่อคืน เราทำงานที่กลับดึกมากๆไม่ได้ทักไปบอกเขาไว้ว่า ถึงบ้านแล้วนะ เงียบไปทั้งคืนเลย เขาก็ถามเรื่องเมื่อคืนว่า หนุ่มไปส่งที่บ้านเป็นไงบ้าง ถามแต่เรื่องของผู้ชายที่มาจีบเราทั้งวัน จนเราเปลี่ยนเรื่องก็ยังกลับมาถามอีกรอบ ไออาการแบบนี้คืออะไรคะ ? ไหนเขาบอกอยากอยู่คนเดียว แต่พอเรามีคนเข้ามา ทำไมเขาถึงมีอาการแบบนี้ มาถามแบบนี้ซ้ำๆ คืออะไรกัน เราไม่อยากคิดอะไรไปเอง ใครพอจะตอบได้บ้างคะ ว่า ไอแบบนี้มันคืออะไร รู้สึกอะไรอยู่"
encoded = tokenizer.encode(text)
print(encoded.ids)
print(encoded.tokens)
print(list(map(lambda x : tokenizer.decode([x]), encoded.ids)))

### If we want to use it again

The Encoding structure exposes multiple properties which are useful when working with transformers models

- normalized_str: The input string after normalization (lower-casing, unicode, stripping, etc.)
- original_str: The input string as it was provided
- tokens: The generated tokens with their string representation
- input_ids: The generated tokens with their integer representation
- attention_mask: If your input has been padded by the tokenizer, then this would be a vector of 1 for any non padded token and 0 for padded ones.
- special_token_mask: If your input contains special tokens such as [CLS], [SEP], [MASK], [PAD], then this would be a vector with 1 in places where a special token has been added.
- type_ids: If your input was made of multiple "parts" such as (question, context), then this would be a vector with for each token the segment it belongs to.
- overflowing: If your input has been truncated into multiple subparts because of a length limit (for BERT for example the sequence length is limited to 512), this will contain all the remaining overflowing parts.

In [17]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("./all-data-wordpiece-30522.tokenizer.json")
encoded =  tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
print(encoded.ids)
print(encoded.tokens)

[1125, 275, 566, 278, 333, 275, 282, 16456, 327, 2565, 368, 317, 830, 340, 274, 301, 334, 301, 13155, 302, 3555, 271, 1303, 1468, 278, 6036, 271, 279, 225, 407, 302, 283, 296, 1273, 444, 271, 19117, 785, 14369, 278, 333, 275, 282, 225, 1986, 271, 12879, 301, 11970, 1474, 793, 1050]
['Ġà¸ªà¸§', 'à¸±', 'à¸ªà¸Ķ', 'à¸µ', 'à¸Ħà¸£', 'à¸±', 'à¸ļ', 'Ġà¸ľà¸¡à¸Ĭ', 'à¸·à¹Ī', 'à¸Ńà¹Ħ', 'à¸Ļà¸Ĺ', 'à¹Į', 'Ġà¸ķà¸Ńà¸Ļà¸Ļ', 'à¸µà¹ī', 'à¸ģ', 'à¹ĩ', 'à¹Ģà¸Ľ', 'à¹ĩ', 'à¸Ļà¹Ģà¸§à¸¥à¸²à¸Ĺ', 'à¸µà¹Ī', 'à¸ľà¸¡à¸ķ', 'à¹ī', 'à¸Ńà¸ĩà¹Ħà¸Ľ', 'à¹Ĥà¸£à¸ĩà¹Ģà¸£', 'à¸µ', 'à¸¢à¸Ļà¹ģà¸¥', 'à¹ī', 'à¸§', 'Ġ', 'Ġà¸Ļ', 'à¸µà¹Ī', 'à¸Ħ', 'à¸·', 'à¸Ńà¸ģà¸²à¸£', 'à¹Ģà¸§', 'à¹ī', 'à¸Ļà¸§à¸£', 'à¸£à¸Ħ', 'à¸ªà¸Ńà¸ĩà¸Ĺ', 'à¸µ', 'à¸Ħà¸£', 'à¸±', 'à¸ļ', 'Ġ', 'Ġà¸Īà¸°à¹Ħà¸Ķ', 'à¹ī', 'à¸Ńà¸Ńà¸ģà¹Ģà¸Ľ', 'à¹ĩ', 'à¸Ļà¸ªà¸Ńà¸ĩ', 'Ġsp', 'ac', 'es']
